In [3]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 401.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=20
✅ 시뮬레이션 초기화 완료: S=20, Randomness='high', M1=768.00, M2=1202.00


In [4]:
only = gp.Model("only")
only.setParam("MIPGap", 1e-7)
only.setParam("OutputFlag", 0)

a = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="a")
bp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
g = only.addVars(I, T+1, S, vtype=GRB.CONTINUOUS, name="gamma")
gc = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
phi1 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi1")
phi2 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi2")
phi3 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi3")

only.update()

obj = gp.quicksum(P_DA[t] * a[i, t] for i, t in product(range(I), range(T))) \
    + gp.quicksum(1 / S * (P_RT[t, s] * bp[i, t, s] - P_PN[t] * bm[i, t, s]) for i, t, s in product(range(I), range(T), range(S)))

only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - a[i, t] == bp[i, t, s] - bm[i, t, s] + gc[i, t, s] - gd[i, t, s])
    only.addConstr(bp[i, t, s] <= R[i, t, s])
    only.addConstr(g[i, t + 1, s] == g[i, t, s] + gc[i, t, s] - gd[i, t, s])
    only.addConstr(gd[i, t, s] <= g[i, t, s])
    only.addConstr(gc[i, t, s] <= K[i] - g[i, t, s])
    only.addConstr(bp[i, t, s] <= M1 * phi3[i, t, s])
    only.addConstr(bm[i, t, s] <= M1 * (1 - phi3[i, t, s]))
    only.addConstr(bm[i, t, s] <= M1 * phi2[i, t, s])
    only.addConstr(gc[i, t, s] <= M1 * (1 - phi2[i, t, s]))
    only.addConstr(gc[i, t, s] <= M1 * phi1[i, t, s])
    only.addConstr(gd[i, t, s] <= M1 * (1 - phi1[i, t, s]))
for i, s in product(range(I), range(S)):
    only.addConstr(g[i, 0, s] == K0[i])

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

a_vals = np.array([[a[i, t].X for t in range(T)] for i in range(I)])
bp_vals = np.array([[[bp[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
bm_vals = np.array([[[bm[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
g_vals  = np.array([[[g[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
gc_vals = np.array([[[gc[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
gd_vals = np.array([[[gd[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-09
Set parameter MIPGap to value 1e-07
Optimal solution found! Objective value: 792691.3336671868


In [5]:
set = gp.Model("set")
set.setParam("MIPGap", 1e-7)
set.setParam("OutputFlag", 0)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")

yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = set.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")

p1 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p1")
p2 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p2")
p3 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p3")
p4 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p4")

set.update()

obj = gp.quicksum(P_DA[t] * gp.quicksum(x[i, t] for i in range(I)) for t in range(T)) + gp.quicksum((1 / S) * (P_RT[t, s] * gp.quicksum(ep[i, t, s] for i in range(I)) - P_PN[t] * gp.quicksum(em[i, t, s] for i in range(I))) for t in range(T) for s in range(S))

set.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(zd[i, t, s] <= z[i, t, s])
    set.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    set.addConstr(yp[i, t, s] <= M1 * p3[i, t, s])
    set.addConstr(ym[i, t, s] <= M1 * (1 - p3[i, t, s]))
    set.addConstr(ym[i, t, s] <= M1 * p2[i, t, s])
    set.addConstr(zc[i, t, s] <= M1 * (1 - p2[i, t, s]))
    set.addConstr(zc[i, t, s] <= M1 * p1[i, t, s])
    set.addConstr(zd[i, t, s] <= M1 * (1 - p1[i, t, s]))
    set.addConstr(z[i, t, s] <= K[i])
    set.addConstr(z[i, t + 1, s] == z[i, t, s] + zc[i, t, s] - zd[i, t, s])
for i, s in product(range(I), range(S)):
    set.addConstr(z[i, 0, s] == K0[i])

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(d[i, j, t, s] for j in range(I)))
    set.addConstr(em[i, t, s] == ym[i, t, s] - gp.quicksum(d[j, i, t, s] for j in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M2 * p4[i, t, s])
    set.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M2 * (1 - p4[i, t, s]))
    set.addConstr(d[i, i, t, s] == 0)

set.optimize()

if set.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {set.objVal}")
else:
    print("No optimal solution found.")
    
x_vals = np.array([[x[i, t].X for t in range(T)] for i in range(I)])
yp_vals = np.array([[[yp[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
ym_vals = np.array([[[ym[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
z_vals  = np.array([[[z[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zc_vals = np.array([[[zc[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zd_vals = np.array([[[zd[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
ep_vals = np.array([[[ep[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
em_vals = np.array([[[em[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
d_vals = np.array([[[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)] for i in range(I)])

Set parameter MIPGap to value 1e-07
Optimal solution found! Objective value: 799045.5658218281


In [6]:
# 시간별 정산
records = []

for s, t, i in product(range(0, 3), range(9, 19), range(I)):
    # only 모델: a * P_DA + bp * P_RT - bm * P_PN
    only_profit = (
        P_DA[t] * a_vals[i, t]
        + P_RT[t, s] * bp_vals[i, t, s]
        - P_PN[t] * bm_vals[i, t, s]
    )

    # set 모델: 외부거래 + 내부클래스
    # 내부 정산: [- P_RT * ∑ d_ijt + P_PN * ∑ d_jit]
    internal = (
        -P_RT[t, s] * sum(d_vals[i, j, t, s] for j in range(I) if j != i)
        + P_PN[t] * sum(d_vals[j, i, t, s] for j in range(I) if j != i)
    )
    # 외부 정산: [P_DA * x + P_RT * y+ - P_PN * y-]
    external = (
        P_DA[t] * x_vals[i, t]
        + P_RT[t, s] * yp_vals[i, t, s]
        - P_PN[t] * ym_vals[i, t, s]
    )

    set_profit = external + internal

    records.append({
        "s": s,
        "t": t,
        "i": i,
        "only_profit": round(only_profit, 2),
        "set_profit": round(set_profit, 2),
        "difference": round(set_profit - only_profit, 2)
    })

df_raw = pd.DataFrame(records)
df_mean = (
    df_raw.groupby(["t", "i"])
    [["only_profit", "set_profit", "difference"]]
    .mean()
    .reset_index()
)

pd.DataFrame(df_mean)


,t,i,only_profit,set_profit,difference
0,9,0,0.000,3926.620,3926.620
1,9,1,7196.997,5402.723,-1794.267
2,9,2,760.693,0.000,-760.693
3,10,0,2607.493,0.000,-2607.493
4,10,1,26104.137,25883.300,-220.840
5,10,2,642.157,642.157,0.000
6,11,0,1807.690,1693.367,-114.323
7,11,1,10887.543,10887.543,0.000
8,11,2,3330.287,5701.000,2370.713
9,12,0,33286.927,29204.303,-4082.620


In [7]:
# 참가자 i별 하루치 총 정산 (over all t, average over s)
daily_records = []

for i in range(I):
    only_total = 0
    set_total = 0
    for t in range(T):
        for s in range(S):
            # only 모델 정산
            only_total += (
                P_DA[t] * a_vals[i, t]
                + P_RT[t, s] * bp_vals[i, t, s]
                - P_PN[t] * bm_vals[i, t, s]
            )

            # set 모델 정산
            internal = (
                - P_RT[t, s] * sum(d_vals[i, j, t, s] for j in range(I) if j != i)
                + P_PN[t] * sum(d_vals[j, i, t, s] for j in range(I) if j != i)
            )
            external = (
                P_DA[t] * x_vals[i, t]
                + P_RT[t, s] * yp_vals[i, t, s]
                - P_PN[t] * ym_vals[i, t, s]
            )
            set_total += external + internal

    # 시나리오 평균
    only_total /= S
    set_total /= S

    daily_records.append({
        "i": i,
        "only_day_total": round(only_total, 2),
        "set_day_total": round(set_total, 2),
        "difference": round(set_total - only_total, 2)
    })

df_daily_total = pd.DataFrame(daily_records)

only_total_sum = df_daily_total["only_day_total"].sum()
set_total_sum = df_daily_total["set_day_total"].sum()
total_difference = set_total_sum - only_total_sum

total_row = {
    "i": "Total",
    "only_day_total": round(only_total_sum, 2),
    "set_day_total": round(set_total_sum, 2),
    "difference": round(total_difference, 2)
}

# 기존 df에 추가
df_with_total = pd.concat([
    df_daily_total,
    pd.DataFrame([total_row])
], ignore_index=True)

pd.DataFrame(df_with_total)

,i,only_day_total,set_day_total,difference
0,0,216712.160,213260.920,-3451.240
1,1,407122.010,424243.680,17121.670
2,2,168857.160,161540.960,-7316.200
3,Total,792691.330,799045.560,6354.230


In [8]:
external_profit_by_t = []
internal_profit_by_t = []
total_profit_by_t = []

s_fixed = 1

for t in range(T):
    external = sum(
        x[i, t].X * P_DA[t] +
        yp[i, t, s_fixed].X * P_RT[t, s_fixed] -
        ym[i, t, s_fixed].X * P_PN[t]
        for i in range(I)
    )

    internal = sum(
        (-sum(d[i, j, t, s_fixed].X for j in range(I) if j != i) * P_RT[t, s_fixed] +
         sum(d[j, i, t, s_fixed].X for j in range(I) if j != i) * P_PN[t])
        for i in range(I)
    )

    external_profit_by_t.append(external)
    internal_profit_by_t.append(internal)
    total_profit_by_t.append(external + internal)

print(f"{'t':>2} | {'External':>12} {'Internal':>12} {'Total':>12}")
print("-" * 42)
for t in range(T):
    print(f"{t:>2} | {external_profit_by_t[t]:>12.2f} {internal_profit_by_t[t]:>12.2f} {total_profit_by_t[t]:>12.2f}")

print("-" * 42)
print(f"{'Σ':>2} | {sum(external_profit_by_t):>12.2f} {sum(internal_profit_by_t):>12.2f} {sum(total_profit_by_t):>12.2f}")


 t |     External     Internal        Total
------------------------------------------
 0 |         0.00         0.00         0.00
 1 |         0.00         0.00         0.00
 2 |         0.00         0.00         0.00
 3 |         0.00         0.00         0.00
 4 |         0.00         0.00         0.00
 5 |         0.00         0.00         0.00
 6 |         0.00         0.00         0.00
 7 |         0.00         0.00         0.00
 8 |      3434.72         0.00      3434.72
 9 |     15471.48      1718.36     17189.83
10 |     25321.80         0.00     25321.80
11 |      3517.07         0.00      3517.07
12 |    102351.73         0.00    102351.73
13 |    123061.51         0.00    123061.51
14 |    162067.48      1597.87    163665.36
15 |    131910.44         0.00    131910.44
16 |     73117.98      1289.78     74407.76
17 |     47293.81         0.00     47293.81
18 |     40076.73         0.00     40076.73
19 |     12593.98      5196.43     17790.41
20 |      6599.46      3469.10   

In [9]:
t = 14
s = 2

print(f"[Settlement Decomposition @ t={t}, s={s}]\n")
print(f"{'i':>3} | {'x·DA':>10} {'+y⁺·RT':>10} {'-y⁻·PN':>10} {'EXᵢ':>10} || {'given':>10} {'received':>10} {'INᵢ':>10} || {'EXᵢ+INᵢ':>10} || {'DISAGG':>10}")
print("-" * 123)

total_ext = 0
total_int = 0
total_sum = 0
check_ext = 0
check_int = 0
check_total = 0

for i in range(I):
    # External Settlement
    x_da = x[i, t].X * P_DA[t]
    yp_rt = yp[i, t, s].X * P_RT[t, s]
    ym_pn = ym[i, t, s].X * P_PN[t]
    ex_i = x_da + yp_rt - ym_pn

    # Internal Settlement
    d_ij = sum(d[i, j, t, s].X for j in range(I) if j != i)
    d_ji = sum(d[j, i, t, s].X for j in range(I) if j != i)

    given = d_ij * P_PN[t]
    received = d_ji * P_RT[t, s]
    in_i = given - received

    # Total per participant
    total_i = ex_i + in_i

    # Accumulate
    total_ext += ex_i
    total_int += in_i
    total_sum += total_i
    
    check_total = sum(x[i, t].X * P_DA[t] + ep[i, t, s].X * P_RT[t, s] - em[i, t, s].X * P_PN[t] for i in range(I))
    disagg = a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t]
    disagg_total = sum(a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t] for i in range(I))

    print(f"{i:>3} | {x_da:10.2f} {yp_rt:10.2f} {-ym_pn:10.2f} {ex_i:10.2f} || "
          f"{given:10.2f} {-received:10.2f} {in_i:10.2f} || {total_i:10.2f} || "
          f"{disagg:10.2f} ({total_i - disagg:5.1f})")

print("-" * 123)
print(f"{'Σ':>3} | {'':>10} {'':>10} {'':>10} {total_ext:10.2f} || "
      f"{'':>10} {'':>10} {total_int:10.2f} || {total_sum:10.2f} || "
      f"{disagg_total:10.2f}")

print("-" * 123)
print(f"{'⁋':>3} | {'':>10} {'':>10} {'':>10} {'':>10} || "
      f"{'':>10} {'':>10} {'':>10} || {check_total:10.2f} || ")

[Settlement Decomposition @ t=14, s=2]

  i |       x·DA     +y⁺·RT     -y⁻·PN        EXᵢ ||      given   received        INᵢ ||    EXᵢ+INᵢ ||     DISAGG
---------------------------------------------------------------------------------------------------------------------------
  0 |   19274.66   14117.48      -0.00   33392.14 ||   30570.87      -0.00   30570.87 ||   63963.01 ||   23128.07 (40834.9)
  1 |   70621.08       0.00  -30570.87   40050.21 ||       0.00  -14117.48  -14117.48 ||   25932.73 ||   45342.84 (-19410.1)
  2 |   16114.88       0.00      -0.00   16114.88 ||       0.00      -0.00       0.00 ||   16114.88 ||   27174.11 (-11059.2)
---------------------------------------------------------------------------------------------------------------------------
  Σ |                                    89557.23 ||                         16453.39 ||  106010.62 ||   95645.02
--------------------------------------------------------------------------------------------------------------

In [14]:
internal_price = np.zeros((T, S))

for t, s in product(range(T), range(S)):
    internal_price[t, s] = P_PN[t] - P_RT[t, s]

In [23]:
for t in range(T):
    print(f"시간 {t}")
    for s in range(0, 3):
        print(f"시나리오 {s}: {internal_price[t, s]} ( = {P_PN[t]} - {P_RT[t, s]} )")

시간 0
시나리오 0: 96.53698295086892 ( = 148.6485 - 52.1115170491311 )
시나리오 1: 102.75794695710388 ( = 148.6485 - 45.89055304289613 )
시나리오 2: 85.58999713742298 ( = 148.6485 - 63.05850286257703 )
시간 1
시나리오 0: 25.280151569635848 ( = 129.714 - 104.43384843036415 )
시나리오 1: 22.06435562047419 ( = 129.714 - 107.64964437952581 )
시나리오 2: 47.75193074973862 ( = 129.714 - 81.96206925026138 )
시간 2
시나리오 0: 64.1927784633496 ( = 119.82750000000001 - 55.63472153665042 )
시나리오 1: 50.58521088051687 ( = 119.82750000000001 - 69.24228911948315 )
시나리오 2: 63.90625723285729 ( = 119.82750000000001 - 55.92124276714272 )
시간 3
시나리오 0: 37.18209520992886 ( = 115.55699999999999 - 78.37490479007113 )
시나리오 1: 29.010571696043726 ( = 115.55699999999999 - 86.54642830395626 )
시나리오 2: 50.963894999863456 ( = 115.55699999999999 - 64.59310500013653 )
시간 4
시나리오 0: 44.005581084388695 ( = 115.22550000000001 - 71.21991891561132 )
시나리오 1: 36.525944594433014 ( = 115.22550000000001 - 78.699555405567 )
시나리오 2: 31.3752921745253 ( = 115.2255000